In [0]:
from google.colab import files
files.upload()

In [0]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c 2020-ai-exam-biker

  0% 0.00/17.1k [00:00<?, ?B/s]
100% 17.1k/17.1k [00:00<00:00, 34.4MB/s]
  0% 0.00/3.09k [00:00<?, ?B/s]
100% 3.09k/3.09k [00:00<00:00, 3.24MB/s]
  0% 0.00/56.3k [00:00<?, ?B/s]
100% 56.3k/56.3k [00:00<00:00, 50.3MB/s]


In [0]:
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
torch.manual_seed(1)

In [0]:
xy_train = pd.read_csv('2020.AI.bike-train.csv', header=0, index_col=0)
xy_train.head()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
id,,,,,,,,,,
3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [0]:
x_test = pd.read_csv('2020.AI.bike-test.csv', header=0, index_col=0)
x_test.head()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
id,,,,,,,,,
1655,4,14.6,0.0,0.6,49.0,2000.0,0.014,41.0,27.0
1657,20,21.8,0.0,4.4,40.0,2000.0,0.048,35.0,19.0
1660,22,18.3,0.0,1.0,37.0,970.0,0.032,169.0,38.0
1662,7,13.2,0.0,0.6,85.0,431.0,0.023,50.0,39.0
1663,8,8.9,0.0,0.9,77.0,613.0,0.005,82.0,40.0


In [0]:
print(xy_train.shape)

(1000, 10)


In [0]:
x_train  = xy_train.iloc[:, :9]
x_train.head()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
id,,,,,,,,,
3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0
6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0
7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0
8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0
9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0


In [0]:
y_train = xy_train.iloc[:, 9:]
y_train.head()

,count
id,
3,49.0
6,159.0
7,26.0
8,57.0
9,431.0


In [0]:
x_train.describe()

,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,11.886000,16.829000,0.033000,2.464600,52.012000,1412.057000,0.038480,57.44900,30.244000
std,6.707757,5.196688,0.178726,1.386726,20.288358,575.842011,0.019338,30.79558,14.553428
min,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,12.00000,9.000000
25%,6.000000,13.100000,0.000000,1.400000,36.000000,910.500000,0.024000,36.00000,20.000000
50%,12.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.038000,52.00000,26.000000
75%,18.000000,20.100000,0.000000,3.400000,68.250000,1991.000000,0.051250,70.00000,37.000000
max,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.106000,269.00000,90.000000


5 만 normalize

In [0]:
vis_min = x_train['hour_bef_visibility'].min()
vis_max = x_train['hour_bef_visibility'].max()
vis_diff = vis_max - vis_min

In [0]:
x_train['hour_bef_visibility'] = (x_train['hour_bef_visibility']-vis_min)/vis_diff
x_test['hour_bef_visibility'] = (x_test['hour_bef_visibility']-vis_min)/vis_diff

In [0]:
x_train['hour_bef_visibility'].describe()

count    1000.000000
mean        0.694098
std         0.299606
min         0.000000
25%         0.433143
50%         0.779917
75%         0.995317
max         1.000000
Name: hour_bef_visibility, dtype: float64

In [0]:
y_min = y_train.min()
y_max = y_train.max()
y_diff = y_max - y_min
y_train = (y_train-y_min)/y_diff
y_train.describe()

,count
count,1000.000000
mean,0.255100
std,0.195041
min,0.000000
25%,0.083721
50%,0.230233
75%,0.353488
max,1.000000


In [0]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)
x_test = torch.FloatTensor(x_test)

# 모델

In [0]:
W = torch.zeros((9,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [0]:
nb_epochs = 3000
for epoch in range(1, nb_epochs+1):
    hypothesis = torch.mm(x_train, W) + b
    cost = torch.mean((hypothesis - y_train) ** 2)

    optimizer = optim.SGD([W,b], lr=1e-5)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch == 1 or epoch%300 ==0:
        print('Epoch: {:4d}/{} Cost: {:.6f}'.format(epoch,nb_epochs, cost))

Epoch:    1/3000 Cost: 0.103079
Epoch:  100/3000 Cost: 0.039740
Epoch:  200/3000 Cost: 0.031607
Epoch:  300/3000 Cost: 0.026363
Epoch:  400/3000 Cost: 0.022971
Epoch:  500/3000 Cost: 0.020770
Epoch:  600/3000 Cost: 0.019338
Epoch:  700/3000 Cost: 0.018403
Epoch:  800/3000 Cost: 0.017793
Epoch:  900/3000 Cost: 0.017392
Epoch: 1000/3000 Cost: 0.017129
Epoch: 1100/3000 Cost: 0.016955
Epoch: 1200/3000 Cost: 0.016840
Epoch: 1300/3000 Cost: 0.016764
Epoch: 1400/3000 Cost: 0.016713
Epoch: 1500/3000 Cost: 0.016679
Epoch: 1600/3000 Cost: 0.016655
Epoch: 1700/3000 Cost: 0.016639
Epoch: 1800/3000 Cost: 0.016628
Epoch: 1900/3000 Cost: 0.016620
Epoch: 2000/3000 Cost: 0.016614
Epoch: 2100/3000 Cost: 0.016609
Epoch: 2200/3000 Cost: 0.016605
Epoch: 2300/3000 Cost: 0.016603
Epoch: 2400/3000 Cost: 0.016600
Epoch: 2500/3000 Cost: 0.016598
Epoch: 2600/3000 Cost: 0.016596
Epoch: 2700/3000 Cost: 0.016594
Epoch: 2800/3000 Cost: 0.016593
Epoch: 2900/3000 Cost: 0.016591
Epoch: 3000/3000 Cost: 0.016590


# Prediction

In [0]:
pred = torch.mm(x_test, W) + b
pred[1]

tensor([0.4603], grad_fn=<SelectBackward>)

In [0]:
pred *= int(y_diff)
pred += int(y_min)

In [0]:
pred.shape

torch.Size([327, 1])

In [0]:
submit = pd.read_csv('2020.AI.bike-submission.csv')
submit['id'] = submit['count']
submit

,id,count
0,0.0,0.0
1,1.0,1.0
2,2.0,2.0
3,3.0,3.0
4,4.0,4.0
...,...,...
322,322.0,322.0
323,323.0,323.0
324,324.0,324.0
325,325.0,325.0


In [0]:
for i in range(len(submit)):
    submit['count'][i] = pred[i]
submit.head()

,id,count
0,0.0,62.557232
1,1.0,198.926224
2,2.0,138.017502
3,3.0,44.073727
4,4.0,18.137793


In [0]:
submit['id'] = pd.Series(submit['id'], dtype="Int32")

In [0]:
submit.to_csv('submit.csv', index=False)

In [0]:
! kaggle competitions submit -c 2020-ai-exam-biker -f submit.csv -m 'first Try'

100% 7.03k/7.03k [00:08<00:00, 858B/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 64, in main
    print(out, end='')
UnicodeEncodeError: 'latin-1' codec can't encode characters in position 34-37: ordinal not in range(256)
